In [1]:
import pandas as pd
import spacy
import re
import numpy as np


In [2]:
df = pd.read_pickle('data/typos_annotated.pkl')

In [3]:
typo_df = df[df["Typo Data"].transform(len) > 0].copy()

In [4]:
typo_df["review_text"].iloc[0]

Lekkie i jak jesteś na kuligu to szypko spadasz slabe sanki. NIE POLECAM!!!!!!!!

In [6]:
typo_df.loc[13, "review_text"][1].ent_iob_ == "O"
# in typo_df.loc[13, "review_text"].ents

True

In [27]:
# Temporary fix for the cyrillic characters
typo_df = typo_df[~typo_df["review_text"].astype(str).str.contains("[\u0400-\u04FF]")]

In [28]:
typo_df[typo_df["review_text"].transform(lambda x: len(x.ents)) > 0]

,review_text,sentiment,Typo Data
13,"(Bosch, to, już, nie, ta, sama, marka, ,, co, ...",Negative,"[[Bosha, [Boscha, Busha, Bosza, Bosra, Boska, ..."
19,"(Piekarnik, ,, cytuję, serwisanta, elektrolux,...",Negative,"[[elektrolux, [electrolux, elektro lux, elektr..."
20,"(Produkt, ma, inny, kolor, na, panelu, inny, n...",Negative,"[[AVANS, [AWANS, AVANT, EVANS], 38, 208, 5]]"
27,"(Nie, polecam, tego, modelu, lodówki, ., Lodów...",Negative,"[[Humidity, [Humity], 29, 189, 8], [Fresh, [Fr..."
29,"(Kupiłem, za, namową, sąsiadów, ,, którzy, zac...",Negative,"[[Plastikowość, [Plastikowo, Plastikowy], 227,..."
...,...,...,...
174443,"(Dron, idealny, na, start, z, fpv, ,, jednak, ...",Positive,"[[fpv, [fp, fp v], 5, 24, 3]]"
175458,"(Jeszcze, nie, uzywane, ,, mam, Dębica, Frigo,...",Neutral,"[[uzywane, [używane, urywane, wzywane, umywane..."
177647,"(Najgorszy, telefon, jaki, miałam, ., Na, pocz...",Negative,"[[pare, [parę, pera, parr, par, pre, parea, pa..."
177926,"(Ja, niestety, nie, polecam, Foresto, ., 2, ty...",Negative,"[[Foresto, [Forest, Forestom, Foresta, Foresty..."


In [29]:
typo_df["Typo_Data_Exclude_Ner"] = (
    typo_df
        .groupby(typo_df.index)[["review_text", "Typo Data"]]
        .apply(lambda x: # Select only typos that are not named entities
               [typo_info for typo_info in x.iloc[0, 1] if x.iloc[0, 0][typo_info[2]].ent_iob_ == "O"]
               )
)

/tmp/ipykernel_5143/1801184966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  typo_df["Typo_Data_Exclude_Ner"] = (


In [30]:
named_ents = (
    typo_df
        .groupby(typo_df.index)[["review_text", "Typo Data"]]
        .apply(lambda x: # Select only the named entities
               [typo_info for typo_info in x.iloc[0, 1] if x.iloc[0, 0][typo_info[2]].ent_iob_ != "O"]
               )
        .drop_duplicates()
        .iloc[1:]
)

In [31]:
# A lot of False Positives
ne_set = set(named_ents.transform(lambda x: [y[0] for y in x]).sum())

print(len(ne_set))
ne_set
# Maybe rerunning spacy with lowercase might help.

3510


{'Clound',
 'KRUPS',
 '5/6',
 'DENON',
 'ewangelikanina',
 'Bozitę',
 'analogazeszly',
 'chyle',
 'Gopro',
 'Aniston',
 'Navy',
 'Khadi',
 'Krzysz',
 'ambilight',
 '50Mpix',
 'RX',
 'Scully',
 'sennheiser',
 'AQI',
 'Ogrodosfera.pl',
 'pyte',
 'VPH',
 'EuroAGDRTV',
 'INTENSE',
 'obsludze',
 'Aquarian',
 'saraja',
 'SOR',
 'Bleue',
 "J'adore",
 'ec412',
 'QE85QN90AATXXH',
 "George'a",
 "O'Clock",
 'Iparfem',
 'bakow',
 'Bricoman',
 'suplefit',
 'Issey',
 '5sek',
 'easyshare',
 'Hansgrohe',
 'KWC',
 'ECP',
 '2008r',
 'Audeze',
 'KeF750',
 'orange',
 'iPrint&Scan',
 'Unifying',
 "Reeves'a",
 'smartboksa',
 'BBK',
 'sls',
 'PHYSIOGEL',
 'Discord',
 'Walcze',
 'Stihla',
 'LCD3',
 'astygmatykow',
 'Electra',
 'netflix',
 'Smoby',
 'dodnes',
 'Expert',
 'covida',
 'miałbyć',
 'Wireworldach',
 'Euroster',
 'Krakwet',
 'Muze',
 'Ideal',
 'Xbox360',
 'iParfem',
 'Monsanto',
 'aqua+allergy',
 'faldo.pl',
 'SuperPharmu',
 '4x4',
 'Walkingbot',
 'oceniť',
 'P+',
 'Rockwell',
 'jogfvbnm',
 'Jed',
 '